In [1]:
%%capture
import os

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install --upgrade peft

In [2]:
!git clone https://github.com/2Phuong5Nam4/ChatBotSynthetic.git

Cloning into 'ChatBotSynthetic'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 103 (delta 41), reused 92 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 801.58 KiB | 12.14 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
%cd ChatBotSynthetic

/kaggle/working/ChatBotSynthetic


In [4]:
import os
from training.sft import DatasetLoader
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    device_map = "balanced",
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-07 10:57:04.563730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767783424.919555      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767783425.034086      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767783425.759631      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767783425.759669      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767783425.759672      24 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import diffusers.loaders.ip_adapter because of the following error (look up to see its traceback):
/usr/local/lib/python3.12/dist-packages/xformers/flash_attn_3/_C.so: undefined symbol: _ZNK3c106SymInt22maybe_as_int_slow_pathEv
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to impo

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2026.1.2 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [6]:
!python scripts/dataset_prepare.py

Loading procedure definitions from /kaggle/working/ChatBotSynthetic/data/extracted_procedure.json...
Loaded 5 procedure definitions

Loading synthetic conversations from /kaggle/working/ChatBotSynthetic/data/synthetic_conversations.jsonl...
Found 243 training records
Mapping procedure_id to procedure details and filtering columns...
Saving training data to /kaggle/working/ChatBotSynthetic/data/train.jsonl...
✓ Saved 243 records to /kaggle/working/ChatBotSynthetic/data/train.jsonl

Loading refined conversations from /kaggle/working/ChatBotSynthetic/data/refined_conversations.jsonl...
Found 32 validation records
Mapping procedure_id to procedure details and filtering columns...
Saving validation data to /kaggle/working/ChatBotSynthetic/data/validation.jsonl...
✓ Saved 32 records to /kaggle/working/ChatBotSynthetic/data/validation.jsonl

Dataset preparation completed!
Training set: 243 samples -> /kaggle/working/ChatBotSynthetic/data/train.jsonl
Validation set: 32 samples -> /kaggle/worki

In [7]:
from training.sft import DatasetLoader
dataset_config = {
        "dataset": {
            "train_path": "data/train.jsonl",
            "validation_path": "data/validation.jsonl",
            "format": "json",  # Note: use "json" not "jsonl" for HF datasets
            "message_field": "messages",
            "text_field": "text"
        }
    }
loader = DatasetLoader(dataset_config, tokenizer)
train_dataset = loader.prepare_dataset(split="train")
dev_dataset = loader.prepare_dataset(split="validation")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1754 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [8]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
from unsloth.chat_templates import train_on_responses_only
import copy

def preprocess_labels(example):
    # 1. Render text đầy đủ
    # Đảm bảo example['messages'] đã được tách (split) như bài trước ta bàn
    text = example['text']
    
    # 2. Tokenize toàn bộ text
    tokenized = tokenizer(text, truncation=True, max_length=max_seq_length, padding=False, add_special_tokens=False)
    input_ids = tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    
    # 3. Tìm vị trí bắt đầu của câu trả lời cuối cùng để Mask
    # Chuỗi cần tìm: "<|im_start|>assistant\n<think>"
    # Nhưng ta muốn giữ lại "<think>" trong labels, nên ta tìm "<|im_start|>assistant\n"
    # VÀ chỉ lấy vị trí tìm thấy CUỐI CÙNG trong chuỗi (rfind)
    
    # Để chính xác, ta tìm index của token separator. 
    # Với Qwen/ChatML, assistant header thường được tokenize thành một chuỗi token cụ thể.
    # Cách an toàn nhất là tìm chuỗi text.
    
    separator = "<|im_start|>assistant\n"
    try:
        # Tìm vị trí bắt đầu của câu assistant CUỐI CÙNG
        start_idx = text.rfind(separator)
        
        if start_idx == -1:
            # Không tìm thấy -> Mask toàn bộ (ignore)
            labels = [-100] * len(labels)
        else:
            # Tính toán độ dài của separator để biết điểm bắt đầu thực sự của câu trả lời
            # Lưu ý: start_idx là char index, cần convert sang token index.
            # Việc mapping char index -> token index khá phức tạp.
            
            # CÁCH ĐƠN GIẢN HƠN: Tokenize từng phần
            
            # Tách text thành 2 phần: [Prompt] và [Response]
            # Prompt = Text từ đầu đến hết "<|im_start|>assistant\n" cuối cùng
            prompt_text = text[:start_idx + len(separator)]
            response_text = text[start_idx + len(separator):]
            
            prompt_ids = tokenizer(prompt_text, add_special_tokens=False).input_ids
            response_ids = tokenizer(response_text, add_special_tokens=False).input_ids
            
            # Gộp lại: labels của phần prompt là -100
            final_input_ids = prompt_ids + response_ids
            final_labels = [-100] * len(prompt_ids) + response_ids
            
            # Cắt nếu quá dài
            if len(final_input_ids) > max_seq_length:
                final_input_ids = final_input_ids[:max_seq_length]
                final_labels = final_labels[:max_seq_length]
                
            input_ids = final_input_ids
            labels = final_labels

    except Exception as e:
        labels = [-100] * len(input_ids)

    return {"input_ids": input_ids, "labels": labels, "attention_mask": [1]*len(input_ids)}

# Áp dụng
train_dataset = train_dataset.map(preprocess_labels) 
dev_dataset = dev_dataset.map(preprocess_labels)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 10,
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
        eval_strategy="steps",       # Hoặc "epoch". Kích hoạt đánh giá trên dev_dataset
        eval_steps=100,                
        load_best_model_at_end=True,
    ),
)

space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

Map:   0%|          | 0/1754 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [9]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

print("CUDA cache cleared. VRAM should now be refreshed.")
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 1,754 | Num Epochs = 2 | Total steps = 440
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 43,646,976 of 8,234,382,336 (0.53% trained)


CUDA cache cleared. VRAM should now be refreshed.
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.715100,0.513854
200,0.620600,0.487651
300,0.451200,0.521909
400,0.492100,0.511586


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [10]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [11]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")
from huggingface_hub import login
login(token=hf_token)
model.push_to_hub("2Phuong5Nam4/Heineken_qwen-3-8B_chatbot", token=hf_token)
tokenizer.push_to_hub("2Phuong5Nam4/Heineken_qwen-3-8B_chatbot", token=hf_token)

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/2Phuong5Nam4/Heineken_qwen-3-8B_chatbot


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            